In [17]:
import numpy as np
from typing import List
from dataclasses import dataclass

# @dataclass
# class Chromosome:
#     gene : List[int]
#     phenome : float = 
#     length : int = 0
#     distnace_metric : List[List[int]]

#     def __post_init__(self):
#         self.length = len(self.gene)
#         self.phenome = fitness(self.gene,distance)
    
    

In [36]:
!wget https://raw.githubusercontent.com/nvidia/NeMo/$BRANCH/examples/tts/conf/fastpitch_align_v1.05.yaml

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
from version2.classes import Chromosome, Parameters
from version2.crossover import point_crossover , ordered_crossover
from version2.mutation import mutate, multi_mutate, inversion, scramble
from version2.replacement import replace_firstweak , replace_weakest
from version2.utils import load_metrics, generate_population, parents_selection, create_chromosome, best_gene, pop_stats, create_parameter_list, find_combinations


set xml path

In [19]:
burma_path = './dataset/burma14.xml'
brazil_path = './dataset/brazil58.xml'

burma_metric = load_metrics(burma_path)
brazil_path = load_metrics(brazil_path)

d:\USERS\Exeter\Courses\Module ECMM409 (2023) Nature-Inspired Computation\Nature_inspired\project01\version2\utils.py:17: RuntimeWarning: invalid value encountered in multiply
  weights_metric = np.nan_to_num(np.identity(num_node) * -np.inf, 0.0)


In [20]:
import torch.nn as nn

class GENETIC(nn.Module):
    def __init__(self, distance_metric) -> None:
        super().__init__()
        self.distance_metric = distance_metric
        
    def forward(self, x):
        pass

In [21]:
@dataclass
class Result:
    last_population : List[Chromosome]
    best_candidate : Chromosome
    avg_fitness : List[float]
    max_fitness : List[float]
    stop_generation : int

## TEST CLASS FN

In [22]:
pop1 = generate_population(10, burma_metric)
pop1

[Chromosome(gene=[4, 9, 13, 10, 1, 8, 11, 5, 0, 6, 2, 12, 7, 3], phenome=0.00014074595355383532, length=14),
 Chromosome(gene=[8, 6, 4, 3, 7, 12, 11, 13, 10, 2, 1, 0, 9, 5], phenome=0.00015401201293700908, length=14),
 Chromosome(gene=[10, 4, 0, 12, 11, 1, 6, 2, 3, 5, 7, 8, 9, 13], phenome=0.00014736221632773356, length=14),
 Chromosome(gene=[10, 2, 1, 11, 8, 4, 13, 9, 3, 5, 0, 12, 6, 7], phenome=0.00013271400132714, length=14),
 Chromosome(gene=[13, 9, 11, 1, 6, 12, 8, 5, 0, 10, 4, 2, 7, 3], phenome=0.000128915817970865, length=14),
 Chromosome(gene=[8, 6, 2, 7, 11, 10, 4, 5, 0, 3, 13, 12, 9, 1], phenome=0.0001405678942929435, length=14),
 Chromosome(gene=[11, 0, 2, 8, 9, 4, 5, 12, 13, 10, 6, 3, 7, 1], phenome=0.00014398848092152627, length=14),
 Chromosome(gene=[2, 9, 4, 6, 7, 3, 13, 8, 5, 1, 10, 11, 12, 0], phenome=0.00012874983906270118, length=14),
 Chromosome(gene=[11, 4, 0, 9, 6, 8, 7, 2, 13, 1, 3, 10, 12, 5], phenome=0.00015931177314003505, length=14),
 Chromosome(gene=[7, 3, 8

In [23]:
best_gene(pop1)

Chromosome(gene=[11, 4, 0, 9, 6, 8, 7, 2, 13, 1, 3, 10, 12, 5], phenome=0.00015931177314003505, length=14)

In [24]:
parents = parents_selection(pop1,5)
parents

[Chromosome(gene=[8, 6, 4, 3, 7, 12, 11, 13, 10, 2, 1, 0, 9, 5], phenome=0.00015401201293700908, length=14),
 Chromosome(gene=[4, 9, 13, 10, 1, 8, 11, 5, 0, 6, 2, 12, 7, 3], phenome=0.00014074595355383532, length=14)]

In [25]:
gene1 , gene2 = ordered_crossover(parents[0],parents[1])
offsprings = [create_chromosome(gene,distance_metric=burma_metric) for gene in ordered_crossover(parents[0],parents[1])]
offsprings

[Chromosome(gene=[3, 6, 4, 10, 1, 8, 11, 13, 7, 2, 12, 0, 9, 5], phenome=0.00015323322096230462, length=14),
 Chromosome(gene=[4, 9, 13, 3, 7, 12, 11, 5, 0, 6, 2, 10, 1, 8], phenome=0.00013764624913971095, length=14)]

In [26]:
for offspring in offsprings:
    popz = replace_firstweak(pop1,offspring)
popz

[Chromosome(gene=[4, 9, 13, 10, 1, 8, 11, 5, 0, 6, 2, 12, 7, 3], phenome=0.00014074595355383532, length=14),
 Chromosome(gene=[8, 6, 4, 3, 7, 12, 11, 13, 10, 2, 1, 0, 9, 5], phenome=0.00015401201293700908, length=14),
 Chromosome(gene=[10, 4, 0, 12, 11, 1, 6, 2, 3, 5, 7, 8, 9, 13], phenome=0.00014736221632773356, length=14),
 Chromosome(gene=[4, 9, 13, 3, 7, 12, 11, 5, 0, 6, 2, 10, 1, 8], phenome=0.00013764624913971095, length=14),
 Chromosome(gene=[13, 9, 11, 1, 6, 12, 8, 5, 0, 10, 4, 2, 7, 3], phenome=0.000128915817970865, length=14),
 Chromosome(gene=[8, 6, 2, 7, 11, 10, 4, 5, 0, 3, 13, 12, 9, 1], phenome=0.0001405678942929435, length=14),
 Chromosome(gene=[11, 0, 2, 8, 9, 4, 5, 12, 13, 10, 6, 3, 7, 1], phenome=0.00014398848092152627, length=14),
 Chromosome(gene=[2, 9, 4, 6, 7, 3, 13, 8, 5, 1, 10, 11, 12, 0], phenome=0.00012874983906270118, length=14),
 Chromosome(gene=[11, 4, 0, 9, 6, 8, 7, 2, 13, 1, 3, 10, 12, 5], phenome=0.00015931177314003505, length=14),
 Chromosome(gene=[7, 3

In [27]:
pop1

[Chromosome(gene=[4, 9, 13, 10, 1, 8, 11, 5, 0, 6, 2, 12, 7, 3], phenome=0.00014074595355383532, length=14),
 Chromosome(gene=[8, 6, 4, 3, 7, 12, 11, 13, 10, 2, 1, 0, 9, 5], phenome=0.00015401201293700908, length=14),
 Chromosome(gene=[10, 4, 0, 12, 11, 1, 6, 2, 3, 5, 7, 8, 9, 13], phenome=0.00014736221632773356, length=14),
 Chromosome(gene=[10, 2, 1, 11, 8, 4, 13, 9, 3, 5, 0, 12, 6, 7], phenome=0.00013271400132714, length=14),
 Chromosome(gene=[13, 9, 11, 1, 6, 12, 8, 5, 0, 10, 4, 2, 7, 3], phenome=0.000128915817970865, length=14),
 Chromosome(gene=[8, 6, 2, 7, 11, 10, 4, 5, 0, 3, 13, 12, 9, 1], phenome=0.0001405678942929435, length=14),
 Chromosome(gene=[11, 0, 2, 8, 9, 4, 5, 12, 13, 10, 6, 3, 7, 1], phenome=0.00014398848092152627, length=14),
 Chromosome(gene=[2, 9, 4, 6, 7, 3, 13, 8, 5, 1, 10, 11, 12, 0], phenome=0.00012874983906270118, length=14),
 Chromosome(gene=[11, 4, 0, 9, 6, 8, 7, 2, 13, 1, 3, 10, 12, 5], phenome=0.00015931177314003505, length=14),
 Chromosome(gene=[7, 3, 8

In [28]:
best_gene(popz)

Chromosome(gene=[11, 4, 0, 9, 6, 8, 7, 2, 13, 1, 3, 10, 12, 5], phenome=0.00015931177314003505, length=14)

## TEST SEARCH

In [32]:
from typing import Callable, List, Any

def search(distance_metric: List[List[int]],
        max_gens:int,
        pop_size:int,
        tour_size:int, 
        co_fn:Callable,
        mut_fn:Callable,
        replace_fn:Callable) -> Any:
    # population = generate_population(distance_metric=sample, pop_size=pop_size)
    population = generate_population(population_size=pop_size,
                                     distance_metric=distance_metric)
    avg_fitness = [np.mean([pop.phenome for pop in population])]
    max_fitness = [np.max([pop.phenome for pop in population])]
    best_candidate = best_gene(population=population)
    # best_candidate:Chromosome = population[np.argmax([pop.phenome for pop in population])]
    for i in range(max_gens):
        # parent1 = tournament_selection(population=population, fitness_value=pop_fitness, tournament_size=tour_size)
        # parent1, parent2 = binary_tournament(population=population, fitness_value=pop_fitness, tournament_size=tour_size)
        parents = parents_selection(population=population, tournament_size=tour_size)
        parent1 , parent2 = parents[0], parents[1]
        gene1, gene2 = co_fn(parent1,parent2)
        child1 , child2 = create_chromosome(gene1,distance_metric=distance_metric), create_chromosome(gene2,distance_metric=distance_metric)
        child1, child2 = mut_fn(child1), mut_fn(child2)
        population = replace_fn(population,child1)
        population = replace_fn(population,child2)
        stats = pop_stats(population=population)
        avg_fitness.append(stats[0])
        max_fitness.append(stats[1][1])
        new_best_candidate = best_gene(population=population)

        if new_best_candidate.phenome > best_candidate.phenome:
            best_candidate = new_best_candidate
        if i % 100:
            print("Processing {i} Steps")
        if (i > 2) and (avg_fitness[i] - avg_fitness[i-1] < 1e-5):
            return population , avg_fitness , max_fitness , i ,best_candidate

    return population , avg_fitness , max_fitness , i ,best_candidate

In [33]:
max_generations = [100,1000,10000]
population_sizes = [50,100,200]
tour_size = [5,10,20]
crossover_functions = [point_crossover, ordered_crossover]
mutate_functions = [mutate, multi_mutate, inversion, scramble]
replace_functions = [replace_firstweak, replace_weakest]


In [34]:
# Test search fn
res = search(burma_metric,100,50,5,point_crossover,inversion,replace_firstweak)
res

point 7
point 0
Processing {i} Steps
point 2
Processing {i} Steps
point 12
Processing {i} Steps


([Chromosome(gene=[1, 10, 9, 6, 11, 3, 13, 7, 0, 2, 8, 5, 4, 12], phenome=0.00017537706068046299, length=14),
  Chromosome(gene=[13, 3, 6, 5, 8, 10, 9, 1, 7, 11, 2, 12, 4, 0], phenome=0.00017123287671232877, length=14),
  Chromosome(gene=[2, 3, 13, 1, 4, 0, 11, 8, 10, 9, 5, 6, 12, 7], phenome=0.0001644195988161789, length=14),
  Chromosome(gene=[5, 10, 13, 8, 6, 0, 9, 3, 1, 4, 12, 2, 11, 7], phenome=0.00012362467548522686, length=14),
  Chromosome(gene=[0, 2, 8, 10, 12, 6, 13, 4, 9, 11, 1, 5, 7, 3], phenome=0.0001292824822236587, length=14),
  Chromosome(gene=[7, 8, 2, 10, 3, 5, 0, 4, 1, 11, 9, 13, 12, 6], phenome=0.00012509382036527395, length=14),
  Chromosome(gene=[12, 3, 1, 5, 0, 13, 7, 4, 2, 11, 10, 9, 8, 6], phenome=0.0001449905756125852, length=14),
  Chromosome(gene=[0, 10, 9, 11, 2, 4, 1, 6, 7, 13, 8, 12, 5, 3], phenome=0.00014598540145985403, length=14),
  Chromosome(gene=[3, 11, 13, 1, 12, 7, 9, 4, 2, 8, 5, 6, 0, 10], phenome=0.00014740566037735848, length=14),
  Chromosome(

In [ ]:
combinations = find_combinations(
    max_generations,
    population_sizes,
    tour_size,
    crossover_functions,
    mutate_functions,
    replace_functions
)
len(combinations)

432

In [ ]:
para_list = create_parameter_list(combinations)
len(para_list)

432

In [ ]:
para_list[:4]

[Parameters(max_gen=100, pop_size=50, tour_size=5, cross_fn=<function point_crossover at 0x000001A6EACFE3E0>, mutate_fn=<function mutate at 0x000001A6EACFE660>, replace_fn=<function replace_firstweak at 0x000001A6EACFE8E0>),
 Parameters(max_gen=100, pop_size=50, tour_size=5, cross_fn=<function point_crossover at 0x000001A6EACFE3E0>, mutate_fn=<function mutate at 0x000001A6EACFE660>, replace_fn=<function replace_weakest at 0x000001A6EACFE980>),
 Parameters(max_gen=100, pop_size=50, tour_size=5, cross_fn=<function point_crossover at 0x000001A6EACFE3E0>, mutate_fn=<function multi_mutate at 0x000001A6EACFE700>, replace_fn=<function replace_firstweak at 0x000001A6EACFE8E0>),
 Parameters(max_gen=100, pop_size=50, tour_size=5, cross_fn=<function point_crossover at 0x000001A6EACFE3E0>, mutate_fn=<function multi_mutate at 0x000001A6EACFE700>, replace_fn=<function replace_weakest at 0x000001A6EACFE980>)]